In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('fashion-mnist_train.csv')
data1 = pd.read_csv('fashion-mnist_test.csv')

In [2]:
x = data.drop(['label'], axis=1)
xg = data1.drop(['label'], axis=1)

In [3]:
y = data['label']
yg = data1['label']

In [4]:
x.shape

(60000, 784)

In [5]:
x=np.array(x)
xg=np.array(xg)

In [6]:
xt = x.T
xgt = xg.T

In [7]:
xt.shape

(784, 60000)

In [8]:
xt = xt/255
xgt = xgt/255

In [9]:
y=np.array(y)
yg=np.array(yg)

In [10]:
yt=y.T
ygt=yg.T

In [11]:
m, n = x.shape

In [12]:
def init_params():
    W1 = np.random.randn(25, 784)
    b1 = np.random.rand(25, 1)
    W2 = np.random.randn(10, 25)
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [13]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 100 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [14]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

In [15]:
W1, b1, W2, b2 = gradient_descent(xt,yt, 0.15, 1000)

Iteration:  0
[9 2 5 ... 9 0 5] [2 9 6 ... 8 8 7]
0.06696666666666666
Iteration:  100
[8 5 4 ... 4 6 7] [2 9 6 ... 8 8 7]
0.45516666666666666
Iteration:  200
[5 9 4 ... 8 8 7] [2 9 6 ... 8 8 7]
0.5142333333333333
Iteration:  300
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.5603
Iteration:  400
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.5909
Iteration:  500
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.6091333333333333
Iteration:  600
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.6217833333333334
Iteration:  700
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.6333166666666666
Iteration:  800
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.64145
Iteration:  900
[8 9 2 ... 8 8 7] [2 9 6 ... 8 8 7]
0.6479666666666667


In [16]:
dev_predictions = make_predictions(xgt, W1, b1, W2, b2)
get_accuracy(dev_predictions, ygt)

[0 1 2 ... 8 6 2] [0 1 2 ... 8 8 1]


0.6474